In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd
import pickle

PROJECT_DIR = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.insert(0, PROJECT_DIR)
sys.path

['/home/uoneway/Project/kss',
 '/home/uoneway/miniconda3/envs/base_p36/lib/python36.zip',
 '/home/uoneway/miniconda3/envs/base_p36/lib/python3.6',
 '/home/uoneway/miniconda3/envs/base_p36/lib/python3.6/lib-dynload',
 '',
 '/home/uoneway/miniconda3/envs/base_p36/lib/python3.6/site-packages',
 '/home/uoneway/miniconda3/envs/base_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/uoneway/.ipython']

In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import kss

In [3]:
DATA_DIR = "/home/uoneway/Project/KoBertSum/data/raw"

## 예시 데이터 불러오기
[한국어 문서 추출요약 AI 경진대회](https://dacon.io/competitions/official/235671/data/) train set의 abstract summary

In [4]:
with open(DATA_DIR + '/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)

train_df = pd.DataFrame(trains)
train_df.head()

,media,id,article_original,abstractive,extractive
0,당진시대,327827480,[당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,국제신문,339840364,[미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,기호일보,371071597,"[인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아, 국유지로...",16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,대구일보,354806783,[대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다...,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,대구신문,347022773,[국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 ...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


## Test

In [33]:
!pip install varname

In [40]:
# 변수 이름을 가져오기 위한 library
# !pip install varname
from varname import varname, nameof
import pprint

def export(obj, file_name=None, dir_path=None):
    file_name = nameof(obj) if file_name is None else file_name
    
    if dir_path is None:  
        file_path = f'./{file_name}'
    else:  # path를 직접 입력해줬다면, 그 경로로 저장
        dir_path = dir_path[:-1] if dir_path[-1] == '/' else dir_path
        file_path = f'{dir_path}/{file_name}.txt'
    
    with open(file_path, 'w') as f:
        # Prints the nicely formatted dictionary
        pprint.pprint(obj, f)

        
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [8]:
from collections import defaultdict

In [48]:
def check_kss(safe=2, to=None):
    sents_splited_list = []
    last_token_dict= defaultdict(set)

    for abstractive in tqdm(train_df['abstractive'][:to]):  # .values
        sents = kss.split_sentences(abstractive, safe=safe)  #safe = True)
        if len(sents) > 1:
            sents_splited_list.append(sents)
            #print(sents)
            
            for idx in range(len(sents)-1):
                cur_sent_tokens = sents[idx].split()
                
                
                if len(cur_sent_tokens) >= 2 and len(sents[idx+1]) > 0:
                    cur_token = cur_sent_tokens[-1]
                    prev_token = cur_sent_tokens[-2]
                    next_token = sents[idx+1].split()[0]
                
                    last_token_dict[cur_token].add(f'{prev_token} {cur_token} {next_token}')
                    
    return sents_splited_list, last_token_dict

In [60]:
def check_and_export(safe=2):
    sents_splited_list, last_token_dict = check_kss(safe=safe)  # to=1000)

    export(sents_splited_list, f'sents_splited_list_s{safe}.txt', PROJECT_DIR + '/tests/check_results')
    export(last_token_dict, f'last_token_dict_s{safe}.txt')
    return last_token_dict

In [59]:
check_and_export(2)

100%|██████████| 1/1 [00:00<00:00, 240.42it/s]


defaultdict(set, {})

In [46]:
check_and_export(1)

100%|██████████| 42803/42803 [02:47<00:00, 255.90it/s]


defaultdict(set,
            {'보이다': {'모습을 보이다 끝내',
              '태도를 보이다 수차례의',
              '하락세를 보이다 6주만에',
              '하락세를 보이다 지난해'},
             '하이다': {'2019광주세계수영선수권대회 하이다 이빙',
              '2019광주세계수영선수권대회조직위원회는 하이다 이빙',
              '28일까지 하이다 이빙',
              '‘2019광주세계수영선수권대회’ 하이다 이빙',
              '‘백미’인 하이다 이빙',
              '결과, 하이다 이빙이',
              '경영과 하이다 이빙',
              '경영과 하이다 이빙에',
              '관심종목인 하이다 이빙',
              '광주세계수영대회에서 하이다 이빙',
              '달성했으며 하이다 이빙,',
              '먼저 하이다 이빙',
              '보이고 하이다 이빙은',
              '부다페스트대회 하이다 이빙에서',
              '수구, 하이다 이빙',
              '없는 하이다 이빙',
              '여자 하이다 이빙',
              '유래한 하이다 이빙',
              '있는 하이다 이빙과',
              '조선대 하이다 이빙',
              '종목인 하이다 이빙',
              '주관하는 하이다 이빙',
              '중 하이다 이빙이',
              '채택된 하이다 이빙',
              '출신의 하이다 이빙'},
             '도주하다': {'끊고 도주하다 광주'},
             '말했다.': {"'축구신동'이라고 말했다. '", '심각

In [49]:
check_and_export(0)

100%|██████████| 42803/42803 [02:54<00:00, 245.39it/s]


defaultdict(set,
            {'보이다': {'모습을 보이다 끝내',
              '태도를 보이다 수차례의',
              '하락세를 보이다 6주만에',
              '하락세를 보이다 지난해'},
             '주요': {'2021년까지 주요 간선도로에',
              '25일부터 주요 벚꽃명소를',
              'WHO에서 주요 폐암',
              '감소로 주요 수출',
              '강화와 주요 행사',
              '같은 주요 산업에선',
              '것이 주요 원인으로',
              '경기도 주요 복지정책',
              '경북도지사는 주요 현안사업에',
              '공공기관은 주요 현안',
              '공약과 주요 사업계획을',
              '광주시교육청은 주요 교육현안과',
              '광주시는 주요 사업을',
              '구성과 주요 사안의',
              '국회의원측이 주요 현안사업과',
              '군위군은 주요 관광지',
              '기관들이 주요 수출국에',
              '기점으로 주요 지역',
              '내년도 주요 핵심',
              '대응과 주요 SOC',
              '대책을 주요 과제로',
              '도 주요 사업으로',
              '등을 주요 내용으로',
              '등을 주요 사업으로',
              '따라 주요 발생연령대인',
              '몰리는 주요 지점에서',
              '미세먼지가 주요 사회문제로',
              '방문객들이 주요 경기장,',
              '배우자로 주요 학대

In [12]:
text = """안녕하십니까? 삼성전자가 11월 13일 삼성전자 서초사옥 다목적홀 5층에서 진행되는 2013 S'데이 멤버십 블루 강연회 "Challenge BLUE, 박찬호&이동우의 삶과 도전" 멤버십 블루 고객 480명을 초청한다.강연회는 삼성전자 멤버십 블루 회원들을 위해 마련된 고객 혜택 행사로 한국인 최초의 메이저리거 박찬호와 시각장애 개그맨 이동우를 초청, 그들의 삶 속에서 펼쳐진 다양한 도전기를 들을 수 있도록 마련했다."""
kss.split_sentences(text, level=0)#safe = True)

['안녕하십니까?',
 '삼성전자가 11월 13일 삼성전자 서초사옥 다목적홀 5층에서 진행되는 2013 S\'데이 멤버십 블루 강연회 "Challenge BLUE, 박찬호&이동우의 삶과 도전" 멤버십 블루 고객 480명을 초청한다.',
 '강연회는 삼성전자 멤버십 블루 회원들을 위해 마련된 고객 혜택 행사로 한국인 최초의 메이저리거 박찬호와 시각장애 개그맨 이동우를 초청, 그들의 삶 속에서 펼쳐진 다양한 도전기를 들을 수 있도록 마련했다.']

In [11]:
text = """이만희 자유한국당 의원이 4일 "청와대 회의에서 대통령의 국회 탓, 야당 탓이 연일 이어지고 있다'며 대통령의 """
kss.split_sentences(text, level=0)#safe = True)

['이만희 자유한국당 의원이 4일 "청와대 회의에서 대통령의 국회 탓, 야당 탓이 연일 이어지고 있다', "'며 대통령의"]

In [18]:
def aa(safe: int = 1):
    print("dd")
    if safe == 1:
        print("aaa")
aa(False)


dd


In [17]:
text = '3일 한국자동차산업협회와 한국수입자동차협회에 따르면 한국지엠(GM)과 르노삼성 등 국내 완성차업체가 모회사의 해외 생산 차량을 수입해 국내에 판매하는 OEM 수입차의 판매량이 2017년까지 하락세를 보이다 지난해 반등했으며 수입차 대비 비중도 높아질 전망이다.'
kss.split_sentences(text, safe=2)

['3일 한국자동차산업협회와 한국수입자동차협회에 따르면 한국지엠(GM)과 르노삼성 등 국내 완성차업체가 모회사의 해외 생산 차량을 수입해 국내에 판매하는 OEM 수입차의 판매량이 2017년까지 하락세를 보이다 지난해 반등했으며 수입차 대비 비중도 높아질 전망이다.']

In [51]:
text1="""광주시장은 "5·18역사왜곡과 폄훼에 대해서도  '5·18역사왜곡 처벌 특별법'을 제정하도록 하겠다"라고 밝혔다."""
text2 = """한 시민은 "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요'라고 말했다."""
text3="""한 시민은 코로나로 인해 '2020년'이란 시간은 멈춘 듯 하다"고 말했다."""
text4="""오늘 5'40 리뷰가 1개 등록됐다."""
kss.split_sentences(text2, safe=2)#safe = True)

['한 시민은 "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요', "'라고 말했다."]

In [58]:
text1 = '국내에 판매하는 OEM 수입차의 판매량이 2017년까지 하락세를 보이다 지난해 반등했으며 수입차 대비 비중도 높아질 전망이다.'
text2 = '전과 8범 A씨는 지난 17일 전자발찌를 끊고 도주하다 붙잡혀 전자발찌 부착기간이 2020년 8월14일까지 늘었다.'
text3 = '국내에 판매하는 OEM 수입차의 판매량은 내년 보다 높아질 전망이다.'

kss.split_sentences(text2, safe=1)

['전과 8범 A씨는 지난 17일 전자발찌를 끊고 도주하다', '붙잡혀 전자발찌 부착기간이 2020년 8월14일까지 늘었다.']